In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import urllib.request
import os

In [6]:
# Set URL
start_time = time.time()
url = """


https://www.cardmarket.com/en/Pokemon/Products/Singles/Team-Rocket/Dark-Dragonair-TR33?language=1,3



"""
cards_list = []

# Selenium settings
option = Options()
option.headless = True
driver = webdriver.Chrome(options=option)

# Initiate Selenium
driver.get(url)

# Load all pages
while True:
    try:
        loadMoreButton = driver.find_element_by_xpath("""//*[@id="loadMoreButton"]""")
        loadMoreButton.click()
        time.sleep(1)
    except Exception as e:
        break

duration_loading = time.time() - start_time
print("Loading of all pages took", round(duration_loading, 1), "seconds to run")

        
# Extract card information
card_name = driver.find_element_by_xpath("/html/body/main/div[3]/div[1]/h1").text.split("\n")[0]
set_name = driver.find_element_by_xpath("/html/body/main/div[3]/div[1]/h1/span").text
number_on_sale = driver.find_element_by_xpath("""//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[6]""").text

# Get all cards
cards = driver.find_elements(By.XPATH, ("""//div[contains(@class, "row no-gutters article-row")]"""))
duration_getting_cards = (time.time() - start_time) - duration_loading

# Loop through each card and add info to list
for e in cards:
    # article id
    article_id = e.get_attribute("id").strip("articleRow")
    
    # image links
    if len(e.find_elements_by_tag_name("a")) == 3:
        image_url = None
    else:
        image_url = e.find_elements_by_tag_name("a")[2].get_attribute("href")
    
    # condition
    condition = e.find_elements_by_tag_name("a")[1].find_elements_by_tag_name("span")[0].get_attribute("data-original-title")
    
    # language
    if e.find_elements_by_tag_name("span")[6].get_attribute("data-original-title") in ["German", "English"]:
        language = e.find_elements_by_tag_name("span")[6].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[7].get_attribute("data-original-title") in ["German", "English"]:
        language = e.find_elements_by_tag_name("span")[7].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[8].get_attribute("data-original-title") in ["German", "English"]:
        language = e.find_elements_by_tag_name("span")[8].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[9].get_attribute("data-original-title") in ["German", "English"]:
        language = e.find_elements_by_tag_name("span")[9].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[10].get_attribute("data-original-title") in ["German", "English"]:
        language = e.find_elements_by_tag_name("span")[10].get_attribute("data-original-title")
        
        
    # edition
    if e.find_elements_by_tag_name("span")[6].get_attribute("data-original-title") in ["First Edition"]:
        edition = e.find_elements_by_tag_name("span")[6].get_attribute("data-original-title")
    if e.find_elements_by_tag_name("span")[7].get_attribute("data-original-title") in ["First Edition"]:
        edition = e.find_elements_by_tag_name("span")[7].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[8].get_attribute("data-original-title") in ["First Edition"]:
        edition = e.find_elements_by_tag_name("span")[8].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[9].get_attribute("data-original-title") in ["First Edition"]:
        edition = e.find_elements_by_tag_name("span")[9].get_attribute("data-original-title")
    elif e.find_elements_by_tag_name("span")[10].get_attribute("data-original-title") in ["First Edition"]:
        edition = e.find_elements_by_tag_name("span")[10].get_attribute("data-original-title")
    else:
        edition = "Second Edition"
        
    # description
    description = e.text.split("\n")
    if len(description) > 4:
        price = description[3].split(" ")
        price = price[0].replace(".", "")
        price = price.replace(",", ".")
        price = float(price)
        
        cards_list.append({"id": article_id, "name": card_name, "edition": edition, "set": "", "price": price, "seller": description[2], "items_sold_by_seller": int(description[0])*1000, "image": image_url, "language": language, "condition": condition})
    else:
        price = description[2].split(" ")
        price = price[0].replace(".", "")
        price = price.replace(",", ".")
        price = float(price)
    

    # Download photos    
    if image_url:
        path_name = f"cards/cardmarket/{card_name}/{edition}/{price}"
        if not os.path.isdir(path_name):
            os.makedirs(path_name)
        urllib.request.urlretrieve(str(image_url), f"{path_name}/{article_id}.jpg")
        
    

        cards_list.append({"id": article_id, "name": card_name, "edition": edition, "set": "", "price": price, "seller": description[1], "items_sold_by_seller": int(description[0]), "image": image_url, "language": language, "condition": condition})
        
        
duration_looping_cards = (time.time() - start_time) - duration_getting_cards - duration_loading - duration_getting_cards
print("Looping through all cards took", round(duration_looping_cards, 1), "seconds to run")


Loading of all pages took 10.1 seconds to run
Looping through all cards took 226.9 seconds to run


In [7]:
final_df = pd.read_csv("pokemon.csv")
df = pd.DataFrame(cards_list)
final_df = final_df.append(df, ignore_index=True)
final_df.to_csv("pokemon.csv", index = False)

In [4]:
len(final_df)

2128

In [5]:
card_name

'Dark Primeape (TR 43)'